<a href="https://colab.research.google.com/github/brunopdrocha/ml_ibmec/blob/main/03_ML_Predicao_Preco_Venda_Imovel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treinamento e avaliacao de modelos com Pycaret


In [ ]:
#Instalando as bibliotecas nao nativas
!pip install pycaret
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of category-encoders to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 14.1 MB/s eta 0:00:00
   ━

In [ ]:
#Importando Bibliotecas Nativas
#import pycaret
import numpy as np
import pandas as pd


In [ ]:
#Importando csv
df = pd.read_csv('https://raw.githubusercontent.com/alvaroriz/datascience_datasets/main/house_prices.csv')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [ ]:
# Criacao de Classe para analise

from pandas.api.types import is_numeric_dtype
class DataFrameAnalyzer:
    def __init__(self, df):
        """
        Inicializa a classe com o DataFrame fornecido.

        Args:
          df: DataFrame de entrada.
        """
        self.df = df

    def variaveis_constantes(self):
        """
        Identifica variáveis constantes no DataFrame.

        Returns:
          Lista de nomes de colunas com valores constantes.
        """
        variaveis_constantes = []
        for col in self.df.columns:
            if self.df[col].nunique() <= 1:
                variaveis_constantes.append(col)
        return variaveis_constantes

    def variaveis_alta_cardinalidade(self, limiar=0.8):
        """
        Identifica variáveis com alta cardinalidade no DataFrame.

        Args:
          limiar: Limiar para alta cardinalidade (padrão 0.8).

        Returns:
          Lista de nomes de colunas com alta cardinalidade.
        """
        variaveis_alta_cardinalidade = []
        for col in self.df.columns:
            if self.df[col].nunique() / len(self.df) >= limiar:
                variaveis_alta_cardinalidade.append(col)
        return variaveis_alta_cardinalidade

    def variaveis_alta_correlacao(self, threshold=0.9):
        """
        Identifica pares de variáveis numéricas que possuem alta correlação.

        Args:
          threshold: Limiar para considerar alta correlação (padrão 0.9).

        Returns:
          Lista de tuplas com pares de variáveis altamente correlacionadas.
        """
        correlacao = self.df.select_dtypes(include=[np.number]).corr()
        pares_correlacionados = []

        for i in range(len(correlacao.columns)):
            for j in range(i):
                if abs(correlacao.iloc[i, j]) > threshold:
                    colname1 = correlacao.columns[i]
                    colname2 = correlacao.columns[j]
                    pares_correlacionados.append((colname1, colname2))

        return pares_correlacionados

    def variaveis_valores_ausentes(self):
        """
        Identifica variáveis com valores ausentes no DataFrame.

        Returns:
          Dicionário com nomes de colunas e a porcentagem de valores ausentes.
        """
        valores_ausentes = self.df.isnull().sum()
        percentuais_ausentes = (valores_ausentes / len(self.df)) * 100
        variaveis_ausentes = percentuais_ausentes[percentuais_ausentes > 0].sort_values(ascending=False)

        return variaveis_ausentes.to_dict()

    def linhas_duplicadas(self):
        """
        Identifica linhas duplicadas no DataFrame.

        Returns:
          Número de linhas duplicadas.
        """
        return self.df.duplicated().sum()

    def variaveis_variancia_zero(self):
        """
        Identifica variáveis com variância zero no DataFrame.

        Returns:
          Lista de nomes de colunas com variância zero.
        """
        variaveis_variancia_zero = []
        for col in self.df.select_dtypes(include=[np.number]).columns:
            if self.df[col].var() == 0:
                variaveis_variancia_zero.append(col)
        return variaveis_variancia_zero

    def alta_quantidade_valores_ausentes(self, threshold=50):
        """
        Identifica variáveis com alta quantidade de valores ausentes no DataFrame.

        Args:
          threshold: Limiar para considerar alta quantidade de valores ausentes (padrão 50%).

        Returns:
          Lista de nomes de colunas com alta quantidade de valores ausentes.
        """
        valores_ausentes = self.df.isnull().sum() / len(self.df) * 100
        return valores_ausentes[valores_ausentes > threshold].index.tolist()

    def variaveis_assimetria(self):
        """
        Identifica variáveis numéricas com alta assimetria (skewness) no DataFrame.

        Returns:
          Dicionário com nomes de colunas e seus respectivos valores de skewness.
        """
        skewness = self.df.select_dtypes(include=[np.number]).apply(lambda x: x.skew()).dropna()
        skewness = skewness[abs(skewness) > 1]  # Considerando alta assimetria se skewness > 1 ou < -1
        return skewness.to_dict()

    def variaveis_mono(self):
        """
        Identifica variáveis monotônicas no DataFrame.

        Returns:
          Lista de nomes de colunas monotônicas.
        """
        monotonic_cols = []
        for col in self.df.select_dtypes(include=[np.number]).columns:
            if self.df[col].is_monotonic_increasing or self.df[col].is_monotonic_decreasing:
                monotonic_cols.append(col)
        return monotonic_cols

    def variaveis_rejeitadas(self):
        """
        Identifica variáveis rejeitadas no DataFrame, como constantes ou com alta quantidade de valores ausentes.

        Returns:
          Lista de nomes de colunas rejeitadas.
        """
        rejeitadas = self.variaveis_constantes()
        rejeitadas += self.alta_quantidade_valores_ausentes()
        return rejeitadas

    def alta_quantidade_infinidades(self):
        """
        Identifica variáveis com alta quantidade de valores infinitos no DataFrame.

        Returns:
          Dicionário com nomes de colunas e a quantidade de valores infinitos.
        """
        infinitos = self.df.replace([np.inf, -np.inf], np.nan)
        num_infinities = infinitos.isnull().sum() - self.df.isnull().sum()
        return num_infinities[num_infinities > 0].to_dict()

    def clusters_alta_correlacao(self, threshold=0.9):
        """
        Identifica agrupamentos de variáveis numéricas com alta correlação no DataFrame.

        Args:
          threshold: Limiar para considerar alta correlação (padrão 0.9).

        Returns:
          Lista de listas de variáveis que estão altamente correlacionadas entre si.
        """
        correlacao = self.df.select_dtypes(include=[np.number]).corr()
        clusters = []
        visited = set()

        for i in range(len(correlacao.columns)):
            cluster = []
            for j in range(i):
                if abs(correlacao.iloc[i, j]) > threshold:
                    colname1 = correlacao.columns[i]
                    colname2 = correlacao.columns[j]
                    if colname1 not in visited and colname2 not in visited:
                        cluster.extend([colname1, colname2])
                        visited.update([colname1, colname2])
            if cluster:
                clusters.append(cluster)

        return clusters

    def resumo_dataset(self):
        """
        Gera um resumo das características do DataFrame.

        Returns:
          Dicionário com informações sobre o DataFrame.
        """
        num_linhas = self.df.shape[0]
        num_colunas = self.df.shape[1]
        variaveis_numericas = [coluna for coluna in self.df.columns if is_numeric_dtype(self.df[coluna])]

        return {
            'Número de linhas': num_linhas,
            'Número de colunas': num_colunas,
            'Variáveis numéricas': variaveis_numericas,
            'Alertas': '-' * 50,
            'Variáveis com valores constantes': self.variaveis_constantes(),
            'Variáveis com alta cardinalidade': self.variaveis_alta_cardinalidade(),
            'Pares de variáveis com alta correlação': self.variaveis_alta_correlacao(),
            'Variáveis com valores ausentes': self.variaveis_valores_ausentes(),
            'Linhas duplicadas': self.linhas_duplicadas(),
            'Variáveis com variância zero': self.variaveis_variancia_zero(),
            'Variáveis com alta quantidade de valores ausentes': self.alta_quantidade_valores_ausentes(),
            'Variáveis com alta assimetria (skewness)': self.variaveis_assimetria(),
            'Variáveis monotônicas': self.variaveis_mono(),
            'Variáveis rejeitadas': self.variaveis_rejeitadas(),
            'Variáveis com alta quantidade de valores infinitos': self.alta_quantidade_infinidades(),
            'Agrupamentos de alta correlação': self.clusters_alta_correlacao()
        }

# Exemplo de uso:
# df = pd.read_csv('seu_arquivo.csv')
# analyzer = DataFrameAnalyzer(df)
# resumo = analyzer.resumo_dataset()

# Print Inteiro
# print (resumo)

# Loop (melhor o de baixo)
# for key, value in resumo.items():
#    print(f"{key}: {value}")



In [ ]:
#Mostrando Caracteristicas do meu df
analyzer = DataFrameAnalyzer(df)
resumo = analyzer.resumo_dataset()
for key, value in resumo.items():
    print(f"{key}: {value}")


Número de linhas: 1460
Número de colunas: 81
Variáveis numéricas: ['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']
Alertas: --------------------------------------------------
Variáveis com valores constantes: []
Variáveis com alta cardinalidade: ['Id']
Pares de variáveis com alta correlação: []
Variáveis com valores ausentes: {'PoolQC': 99.52054794520548, 'MiscFeature': 96.30136986301369, 'Alley': 93.76712328767123, 'Fence': 80.75342465753424, 'MasVnrType': 59.726027397260275, 'FireplaceQu': 47.26027397260274, 'LotFrontage'

In [ ]:
df.drop(['Id','Alley', 'MasVnrType', 'PoolQC', 'Fence', 'MiscFeature'], axis=1,inplace=True)

In [ ]:
df

## Separando nossa nase de Teste e Treinamento

In [ ]:
#Dados de Treinamento
base_treinamento = df.sample(frac=0.8, random_state=42)

#Dados de Teste para certificamos a qualidade do nosso modelo ao final do trabalho
base_teste = df.drop(base_treinamento.index)

print(f'Base de Treinamento: {base_treinamento.shape}')
print(f'Base de Teste: {base_teste.shape}')

In [ ]:
# Realizar a configuracao do Pycaret
from pycaret.regression import setup
s = setup(data=base_treinamento,
          target = 'SalePrice',
          session_id=42,
          normalize=True,
          remove_outliers=True,
          use_gpu=True,
          train_size=0.8,
          fold=10,
          n_jobs=-1,
          log_experiment=True,
          experiment_name='house_prices_experiment',
          remove_multicollinearity=True,
          multicollinearity_threshold=0.95,
          feature_selection=False)



In [ ]:
from pycaret.regression import compare_models
modelos = compare_models(fold=10)

In [ ]:
from pycaret.regression import create_model
modelo = create_model('en')

In [ ]:
from pycaret.regression import tune_model
modelo_tunado = tune_model(modelo)

In [ ]:
modelo_tunado

## Realizando a predicao

In [ ]:
from pycaret.regression import predict_model
pred = predict_model(modelo_tunado, data=base_teste)

## Analisando o modelo


In [ ]:
from pycaret.regression import plot_model
plot_model(modelo_tunado)

In [ ]:
from pycaret.regression import evaluate_model
evaluate_model(modelo_tunado)

## Finalizando modelo


In [ ]:
from pycaret.regression import finalize_model
modelo_final = finalize_model(modelo_tunado)

In [ ]:
from pycaret.regression import save_model
save_model(modelo_final, 'modelo_final')